[View in Colaboratory](https://colab.research.google.com/github/usernaamee/learning-good-representations/blob/master/Exp1-predicting_future_mfcc_frames.ipynb)

In [1]:
!apt install ffmpeg
!pip install librosa

import os
import librosa
import numpy as np
from keras.models import *
from keras.layers import *


%matplotlib inline

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  dbus i965-va-driver libaacs0 libapparmor1 libasound2 libasound2-data libass9
  libasyncns0 libavc1394-0 libavcodec57 libavdevice57 libavfilter6
  libavformat57 libavresample3 libavutil55 libbdplus0 libbluray2
  libboost-filesystem1.62.0 libboost-system1.62.0 libbs2b0 libcaca0 libcairo2
  libcapnp-0.5.3 libcdio-cdda1 libcdio-paranoia1 libcdio13 libchromaprint1
  libcrystalhd3 libdbus-1-3 libdc1394-22 libdrm-amdgpu1 libdrm-common
  libdrm-intel1 libdrm-nouveau2 libdrm-radeon1 libdrm2 libegl1-mesa libelf1
  libfftw3-double3 libflac8 libflite1 libfribidi0 libgbm1 libgl1-mesa-dri
  libgl1-mesa-glx libglapi-mesa libgme0 libgraphite2-3 libgsm1 libharfbuzz0b
  libiec61883-0 libjack-jackd2-0 libllvm5.0 libmirclient9 libmircommon7
  libmircore1 libmirprotobuf3 libmp3lame0 libmpg123-0 libnuma1 libogg0
  libopenal-data libopenal1 libopencv-core3.1 l

Using TensorFlow backend.


In [2]:
!wget http://www.openslr.org/resources/31/dev-clean-2.tar.gz

--2018-09-05 08:09:14--  http://www.openslr.org/resources/31/dev-clean-2.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126046265 (120M) [application/x-gzip]
Saving to: ‘dev-clean-2.tar.gz’

dev-clean-2.tar.gz  100%[===================>] 120.21M   131MB/s    in 0.9s    

2018-09-05 08:09:15 (131 MB/s) - ‘dev-clean-2.tar.gz’ saved [126046265/126046265]



In [3]:
!tar xvfz dev-clean-2.tar.gz

LibriSpeech/dev-clean-2/
LibriSpeech/dev-clean-2/5694/
LibriSpeech/dev-clean-2/5694/64038/
LibriSpeech/dev-clean-2/5694/64038/5694-64038.trans.txt
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0000.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0001.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0002.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0003.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0004.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0005.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0006.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0007.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0008.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0009.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0010.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0011.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0012.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0013.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0014.flac
LibriSpeech/d

In [4]:
import os
import glob

basedir = "LibriSpeech/dev-clean-2/"
files = glob.glob(basedir+'**/*.flac', recursive=True)

print(len(files))

1089


In [5]:
from tqdm import tqdm 

audios_5s = [librosa.load(f, sr=16000, mono=True, duration=5.0) for f in tqdm(files)]

100%|██████████| 1089/1089 [02:19<00:00,  7.80it/s]


In [6]:
feats_13d = [librosa.feature.mfcc(a[0], sr=a[1], n_fft=400, hop_length=160, n_mels=80, n_mfcc=13) for a in tqdm(audios_5s)]

100%|██████████| 1089/1089 [00:11<00:00, 91.22it/s]


In [0]:
def get_dataset(feats, n_past_steps=1, nth_future_step=1, n_examples=50000):
  total_feats = np.sum([f.shape[1] for f in feats])
  skip_len = total_feats // n_examples
  print('# examples (desired): {}\tSkip len: {}'.format(n_examples, skip_len))
  X = []
  Y = []
  for feat in feats:
    feat = feat.T
    for i in range(n_past_steps, len(feat) - n_past_steps - nth_future_step - skip_len, skip_len):
      x = feat[i-n_past_steps:i]
      y = feat[i+nth_future_step-1]
      X.append(x)
      Y.append(y)
  print('# examples (actual): {}'.format(len(X)))
  return np.array(X), np.array(Y)

In [12]:
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=0)

n_hidden_layers = [1, 2]
units_list = [32, 256]

for fut in range(1,10):
  for past in range(1, 20):
    X, Y = get_dataset(feats_13d, past, 1, n_examples=50000)
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y)
    for nl in n_hidden_layers:
      for n_units in units_list:
        model = Sequential()
        model.add(Flatten(input_shape=X.shape[1:]))
        for _ in range(nl):
          model.add(Dense(n_units, activation='relu'))
        model.add(Dense(Y.shape[1], activation='linear'))
        model.compile(loss='mse', optimizer='adam')
        model.fit(Xtrain, Ytrain, validation_split=0.1, epochs=100, batch_size=128, verbose=0, callbacks=[es])
        loss = model.evaluate(Xtest, Ytest, verbose=0)
        print('past: {} future: {}\tlayers: {} units: {}\ttest-loss: {}'.format(past, fut, nl, n_units, loss))

# examples (desired): 50000	Skip len: 9
# examples (actual): 51314
past: 1 future: 1	layers: 1 units: 32	test-loss: 142.92961955027752
past: 1 future: 1	layers: 1 units: 256	test-loss: 140.42962288772875
past: 1 future: 1	layers: 2 units: 32	test-loss: 143.07824983150033
past: 1 future: 1	layers: 2 units: 256	test-loss: 145.03678385694192
# examples (desired): 50000	Skip len: 9
# examples (actual): 51201
past: 2 future: 1	layers: 1 units: 32	test-loss: 123.29091035950026
past: 2 future: 1	layers: 1 units: 256	test-loss: 119.17757246686317
past: 2 future: 1	layers: 2 units: 32	test-loss: 124.03633942182023
past: 2 future: 1	layers: 2 units: 256	test-loss: 120.22644428580676
# examples (desired): 50000	Skip len: 9
# examples (actual): 50481
past: 3 future: 1	layers: 1 units: 32	test-loss: 123.61722357897814
past: 3 future: 1	layers: 1 units: 256	test-loss: 122.29651024590751
past: 3 future: 1	layers: 2 units: 32	test-loss: 127.01407690269616
past: 3 future: 1	layers: 2 units: 256	test-lo

KeyboardInterrupt: ignored

In [14]:
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=0)

n_hidden_layers = [2, 4]
units_list = [128, 256]

for fut in range(1,10):
  for past in range(15, 25):
    X, Y = get_dataset(feats_13d, past, 1, n_examples=50000)
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y)
    for nl in n_hidden_layers:
      for n_units in units_list:
        model = Sequential()
        model.add(Flatten(input_shape=X.shape[1:]))
        for _ in range(nl):
          model.add(Dense(n_units, activation='relu'))
        model.add(Dense(Y.shape[1], activation='linear'))
        model.compile(loss='mse', optimizer='adam')
        model.fit(Xtrain, Ytrain, validation_split=0.1, epochs=100, batch_size=128, verbose=0, callbacks=[es])
        loss = model.evaluate(Xtest, Ytest, verbose=0)
        print('past: {} future: {}\tlayers: {} units: {}\ttest-loss: {}'.format(past, fut, nl, n_units, loss))

# examples (desired): 50000	Skip len: 9
# examples (actual): 47982
past: 15 future: 1	layers: 2 units: 128	test-loss: 138.98125999631307
past: 15 future: 1	layers: 2 units: 256	test-loss: 315.1824159281935
past: 15 future: 1	layers: 4 units: 128	test-loss: 139.27900093640832
past: 15 future: 1	layers: 4 units: 256	test-loss: 239.97066261975834
# examples (desired): 50000	Skip len: 9
# examples (actual): 47279
past: 16 future: 1	layers: 2 units: 128	test-loss: 131.9981208517265
past: 16 future: 1	layers: 2 units: 256	test-loss: 136.18135478734567
past: 16 future: 1	layers: 4 units: 128	test-loss: 131.24900847304292
past: 16 future: 1	layers: 4 units: 256	test-loss: 129.52764686933023
# examples (desired): 50000	Skip len: 9
# examples (actual): 47165
past: 17 future: 1	layers: 2 units: 128	test-loss: 127.47692477719269
past: 17 future: 1	layers: 2 units: 256	test-loss: 128.36989788122708
past: 17 future: 1	layers: 4 units: 128	test-loss: 140.8291951028107
past: 17 future: 1	layers: 4 uni